In [1]:
import os
import sys
import numpy as np
import pandas as pd

In [2]:
os.chdir('/labs/mmdavis/chen/RNA-seq/tpm')
sys.path.append('/labs/mmdavis/chen/code/bulk_RNA')

In [ ]:
from megha_rna.megha_rna import *

In [5]:
import os
from functools import reduce

import pandas as pd

os.chdir('/labs/mmdavis/chen/RNA-seq/tpm')
sample_list = [
    "alveolar_mac_1", "alveolar_mac_2", "alveolar_mac_3", "alveolar_mac_4", "alveolar_mac_5", "alveolar_mac_6",
    "gdT_d1_1", "gdT_d1_2", "gdT_d3_1", "gdT_d3_2", "gdT_d3_3", "gdT_d7_1", "gdT_d7_2", "gdT_d7_3", "gdT_LN_1",
    "gdT_LN_2", "gdT_LN_3", "gdT_LN_4", "KO_epithelial_1", "KO_epithelial_2", "KO_epithelial_3", "KO_epithelial_4",
    "KO_remaining_1", "KO_remaining_2", "TCRd_epithelial_1", "WT_epithelial_1", "WT_epithelial_2", "WT_epithelial_3",
    "WT_epithelial_4", "WT_epithelial_5", "WT_epithelial_6", "WT_remaining_1", "WT_remaining_2"
]

sample_unique_list = ['alveolar_mac', 'gdT_d1', 'gdT_d3', 'gdT_d7', 'gdT_LN', 'KO_epithelial',
                      'KO_remaining', 'TCRd_epithelial', 'WT_epithelial', 'WT_remaining']


def sample_rename():
    sample_rename_list = []
    for i in sample_list:
        for j in range(len(sample_unique_list)):
            name = i.split('_')
            sample_name = name[0] + "_" + name[1]
            sample_no = name[2]
            if sample_name == sample_unique_list[j]:
                new_sample_name = 'sample' + str(j)+'_'+sample_no
                sample_rename_list.append(new_sample_name)


def get_all_gene():
    All_genes = []
    for i in range(len(sample_list)):
        df = pd.read_csv(sample_list[i] + '.csv', sep='\t')
        df = df.loc[:, ['Gene_Id', 'TPM']]
        gene_list = list(df.Gene_Id)
        All_genes.extend(gene_list)
    all_genes = list(set(All_genes))
    df0=pd.DataFrame(all_genes, columns=['Gene_Id'])
    return df0 


def combine_tpm():
    df0 = get_all_gene()
    dfs = []
    for i in range(len(sample_list)):
        df = pd.read_csv(sample_list[i] + '.csv', sep='\t')
        df = df.loc[:, ['Gene_Id', 'TPM']]
        df.columns = ['Gene_Id', sample_list[i]]
        df1 = pd.merge(df0, df, on='Gene_Id', how='outer')
        df1 = df1.set_index(['Gene_Id'])
        dfs.append(df1)
    df_all = reduce(lambda x, y: pd.merge(x,y,left_index=True, right_index=True), dfs)
    df_all = df_all.filena(0)
    return df_all
    df_all.to_csv('df_all.csv', index=False)




In [6]:
df0 = get_all_gene()
df0.head()

,Gene_Id
0,Chad
1,P4ha2
2,Usp17la
3,Gm4719
4,Tbx15
...,...
24488,Rnf223
24489,Ccdc92
24490,Ankrd54
24491,Phip


In [55]:

dfs = []
for i in range(len(sample_list)):
    df = pd.read_csv(sample_list[i] + '.csv', sep='\t')
    df = df.loc[:, ['Gene_Id', 'TPM']]
    df.columns = ['Gene_Id', sample_list[i]]
    df1 = df0.merge(df, on='Gene_Id', how='left')
    df1 = df1.groupby(['Gene_Id']).agg('sum')
    #df1 = df1.set_index(['Gene_Id'])
    dfs.append(df1)
df_all = reduce(lambda x, y: pd.concat([x,y],axis=1), dfs)
df_all = df_all.fillna(0)
  
    #df_all.to_csv('df_all.csv', index=False)

In [68]:
df_all.reset_index().to_csv('/labs/mmdavis/chen/code/bulk_RNA/megha_rna/df_all.csv', index=False)

In [66]:
df_all.reset_index()

,Gene_Id,alveolar_mac_1,alveolar_mac_2,alveolar_mac_3,alveolar_mac_4,alveolar_mac_5,alveolar_mac_6,gdT_d1_1,gdT_d1_2,gdT_d3_1,...,KO_remaining_2,TCRd_epithelial_1,WT_epithelial_1,WT_epithelial_2,WT_epithelial_3,WT_epithelial_4,WT_epithelial_5,WT_epithelial_6,WT_remaining_1,WT_remaining_2
0,0610005C13Rik,0.000000,0.267246,0.273999,0.061386,0.129742,0.632002,2.201850,2.366170,2.05681,...,1.81559,2.270300,1.932850,2.322100,0.613214,1.020140,1.901220,1.255230,0.996733,2.16943
1,0610009B22Rik,70.992100,40.736200,41.698000,60.862000,33.889800,43.164600,10.272800,13.498000,10.97870,...,38.87160,61.039900,75.621900,39.371500,26.699500,35.146400,45.133200,35.655700,21.249500,33.64580
2,0610009E02Rik,8.235850,6.669850,5.777990,5.707350,7.579200,9.288280,1.338330,2.260130,14.69480,...,5.87390,2.876390,2.014400,2.163900,0.364786,0.503142,11.956700,8.051660,0.844074,3.60025
3,0610009L18Rik,1.800240,0.989836,9.300840,0.180324,0.000000,11.523200,7.473150,12.623100,3.62515,...,16.59260,10.329800,7.097240,17.851800,7.611260,8.461250,17.491500,16.614400,10.864200,10.10710
4,0610010F05Rik,11.658600,8.429080,5.474290,6.940840,8.269130,7.464960,1.976380,1.444040,8.54413,...,3.75099,4.133590,3.574190,2.784320,0.521693,0.484645,3.676730,2.110970,1.084440,3.81386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24488,Zyx,78.405600,80.996800,70.477500,61.105500,66.533800,96.625500,90.198800,104.049000,96.55880,...,291.83400,71.360100,46.887800,58.158900,10.589300,20.040200,72.567700,78.343500,110.846000,358.12500
24489,Zzef1,13.686600,8.164280,8.200160,9.028800,7.832820,8.194120,3.118670,3.459560,17.66910,...,3.17227,1.718750,0.767399,0.792562,0.134436,0.269290,2.309740,2.824130,0.687366,2.31525
24490,Zzz3,29.688100,18.302500,12.786300,18.168800,14.518300,17.388500,15.813200,14.664200,35.58910,...,8.02955,4.157000,3.067450,1.967890,0.823538,1.014720,8.557790,7.501810,2.229880,9.04129
24491,a,0.362471,0.643715,0.468172,1.107380,0.844108,0.859318,17.317100,16.332800,8.33314,...,4.89196,0.433001,0.304854,0.362502,0.095355,0.131829,0.806927,0.646341,1.699630,3.47888


In [47]:
df0\\\

,Gene_Id
0,Chad
1,P4ha2
2,Usp17la
3,Gm4719
4,Tbx15
...,...
24488,Rnf223
24489,Ccdc92
24490,Ankrd54
24491,Phip


In [38]:
from collections import Counter

Counter(dfs[0].index).most_common(20)

[('Mir684-1', 5),
 ('Mir5098', 4),
 ('Gm5643', 2),
 ('Mir692-3', 2),
 ('Gm2506', 2),
 ('Gm13306', 2),
 ('LOC100861978', 2),
 ('LOC108167347', 2),
 ('Ppp1ccb', 2),
 ('Fau', 2),
 ('Gm5512', 2),
 ('Mir692-2', 2),
 ('Mir684-1#2', 2),
 ('Bc1', 2),
 ('Btg3', 2),
 ('Timm8a1', 2),
 ('Ccl27b', 2),
 ('Gm13305', 2),
 ('Il11ra2', 2),
 ('Ppia', 2)]

In [21]:
set1 = set(df0.Gene_Id)

In [22]:
set2 = set(dfs[0].index)

In [27]:
len(set1) 

24493

In [28]:
len(set2)

24493

In [24]:
set1

{'Chad',
 'P4ha2',
 'Usp17la',
 'Gm4719',
 'Tbx15',
 'Cryab',
 'Nudt19',
 'Pomt2',
 'Gm4750#2',
 'Sycp1',
 'Mnd1',
 'Pmpca',
 'Dsc1',
 'C2cd5',
 'Entpd3',
 'Snord49a',
 'Fhad1',
 'Pip5kl1',
 'Ces2b',
 'Nkapd1',
 'Rsad2',
 'Ankrd29',
 'BC006965',
 '9530082P21Rik',
 'Il16',
 'Cyp2b9',
 'Armc6',
 'Nrxn2',
 'Ccl11',
 'Dgat2l6',
 'Tent5c',
 'Med16',
 'Gm3415#1',
 'Rpe65',
 'Meg3',
 'Casp3',
 'Ndc1',
 'Gm4432',
 'Iqcb1',
 'Btbd3',
 'Invs',
 'Il12a',
 'Tmem161b',
 'Snora44',
 'Tmem42',
 'Vmn1r58',
 'Gm10471',
 'Actb',
 'Prpf40b',
 'Gm5643',
 'Micu2',
 'Lig1',
 'Spata20',
 'Pllp',
 'Ppp1r3b',
 'Scg3',
 'F930015N05Rik',
 'Mgat5b',
 'Zxda',
 'Inha',
 'Mir6968',
 'Rbmy#3',
 'Ncl',
 'Cd99l2',
 'Zim2',
 'Rps6kc1',
 'Timm8b',
 'Uhmk1',
 'Marchf9',
 'Ttpal',
 'Pelo',
 'Chsy1',
 '5430401F13Rik',
 'Cysltr2',
 'Tas2r138',
 'Myo9a',
 '4930452B06Rik',
 'Slc19a2',
 'Hsd3b6',
 'Pcsk2os1',
 'Yy1',
 'Mir6361',
 'Leng8',
 'Vsir',
 'Spata5l1',
 'St8sia3os',
 'Bglap',
 'Cxcl9',
 'Dtx3',
 'Gm21293#1',
 'AI854703'